# **DATA3888 Project: Optiver**

In [1]:
import os
import pandas as pd
import dask.dataframe as dd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
def load_data(directory: str) -> pd.DataFrame:

    all_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".csv"):
                all_files.append(os.path.join(root, file))

    if not all_files:
        raise FileNotFoundError("No CSV files found in the given directory.")

    df = dd.read_csv(all_files)
    return df.compute()

data_path = "./Data/individual_book_train"
df = load_data(data_path)

In [3]:
df.head()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,stock_id
0,5,0,1.000129,1.000386,0.999871,1.000643,302,615,500,400,13
1,5,1,1.000129,1.000386,0.999871,1.000643,602,515,400,500,13
2,5,2,1.000129,1.000386,0.999871,1.000643,502,515,400,500,13
3,5,3,1.000129,1.000386,0.999871,1.000643,502,515,400,500,13
4,5,4,1.000129,1.000386,0.999871,1.000643,502,515,400,600,13
